# RippleNet

In [14]:
import sys
sys.path.append("../../")
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import argparse 
from reco_utils.evaluation.python_evaluation import auc, precision_at_k, recall_at_k
from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_stratified_split

from reco_utils.recommender.ripplenet.preprocess import (read_item_index_to_entity_id_file, 
                                         convert_rating, 
                                         convert_kg)

from reco_utils.recommender.ripplenet.data_loader import (
                                         load_kg, 
                                         get_ripple_set)

from reco_utils.recommender.ripplenet.train import (fit, predict)

from reco_utils.recommender.ripplenet.model import RippleNet

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))

System version: 3.6.8 |Anaconda, Inc.| (default, Dec 29 2018, 19:04:46) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas version: 0.25.1


In [15]:
# Select MovieLens data size: 100k, 1m, 10m, or 20m
MOVIELENS_DATA_SIZE = '100k'
# Ripple parameters
n_epoch = 10 #the number of epochs
batch_size = 1024 #batch size
dim = 16 #dimension of entity and relation embeddings
n_hop = 2 #maximum hops
kge_weight = 0.01 #weight of the KGE term
l2_weight = 1e-7 #weight of the l2 regularization term
lr = 0.02 #learning rate
n_memory = 32 #size of ripple set for each hop
item_update_mode = 'plus_transform' #how to update item at the end of each hop
using_all_hops = True #whether using outputs of all hops or just the last hop when making prediction
#Evaluation parameters
k = 5

## Introduction

> RippleNet: Propagating User Preferences on the Knowledge Graph for Recommender Systems
> Hongwei Wang, Fuzheng Zhang, Jialin Wang, Miao Zhao, Wenjie Li, Xing Xie, Minyi Guo
> The 27th ACM International Conference on Information and Knowledge Management (CIKM 2018)

Online code of RippleNet: https://github.com/hwwang55/RippleNet

To address the sparsity and cold start problem of collaborative filtering, researchers usually make use of side information, such as social networks or item attributes, to improve recommendation performance. This paper considers the knowledge graph as the source of side information. To address the limitations of existing embedding-based and path-based methods for knowledge-graph-aware recommendation, we propose RippleNet, an end-to-end framework that naturally incorporates the knowledge graph into recommender systems. Similar to actual ripples propagating on the water, RippleNet stimulates the propagation of user preferences over the set of knowledge entities by automatically and iteratively extending a user’s potential interests along links in the knowledge graph. The multiple "ripples" activated by a user’s historically clicked items are thus superposed to form the preference distribution of the user with respect to a candidate item, which could be used for predicting the final clicking probability. Through extensive experiments on real-world datasets, we demonstrate that RippleNet achieves substantial gains in a variety of scenarios, including movie, book and news recommendation, over several state-of-the-art baselines.

![alt text](https://github.com/hwwang55/RippleNet/raw/master/framework.jpg)


## Read original data and transform entity ids to numerical

In [16]:
kg_original = pd.read_csv("https://recodatasets.blob.core.windows.net/wikidata/movielens_{}_wikidata.csv".format(MOVIELENS_DATA_SIZE))
ratings_original = movielens.load_pandas_df(MOVIELENS_DATA_SIZE,
                              ('UserId', 'ItemId', 'Rating', 'Timestamp'),
                             title_col='Title',
                             genres_col='Genres',
                             year_col='Year')
rating_threshold = 4

100%|██████████| 4.81k/4.81k [00:01<00:00, 3.04kKB/s]


In [17]:
def transform_id(df, entities_id, col_transform, col_name = "unified_id"):
    df = df.merge(entities_id, left_on = col_transform, right_on = "entity")
    df = df.rename(columns = {"unified_id": col_name})
    return df.drop(columns = [col_transform, "entity"])

In [18]:
var_id = "movielens_id"
entities_id = pd.DataFrame({"entity":list(set(kg_original.original_entity)) + list(set(kg_original.linked_entities))}).reset_index()
entities_id = entities_id.rename(columns = {"index": "unified_id"})

item_to_entity = kg_original[[var_id, "original_entity"]].drop_duplicates().reset_index().drop(columns = "index")
item_to_entity = transform_id(item_to_entity, entities_id, "original_entity")

In [19]:
kg = kg_original[["original_entity", "linked_entities"]].drop_duplicates()
kg = transform_id(kg, entities_id, "original_entity", "original_entity_id")
kg = transform_id(kg, entities_id, "linked_entities", "linked_entities_id")
kg["relation"] = 1
kg_wikidata = kg[["original_entity_id","relation", "linked_entities_id"]]

In [20]:
vars_movielens = ["UserId", "ItemId", "Rating", "Timestamp"]
ratings = ratings_original[vars_movielens].sort_values(vars_movielens[1])

## Preprocess module from RippleNet

In [22]:
item_index_old2new, entity_id2index = read_item_index_to_entity_id_file(item_to_entity)

In [25]:
ratings_final = convert_rating(ratings, item_index_old2new = item_index_old2new,
                               threshold = rating_threshold, seed = 12)

converting rating file ...
number of users: 942
number of items: 1677


In [28]:
kg_final = convert_kg(kg_wikidata, entity_id2index = entity_id2index)

converting kg file ...
number of entities (containing items): 22994
number of relations: 1


## Split Data

In [31]:
train_data, test_eval_data = python_stratified_split(ratings_final, ratio=0.6, col_user='user_index', col_item='item', seed=12)
test_data, eval_data = python_stratified_split(test_eval_data, ratio=0.5, col_user='user_index', col_item='item', seed=12)

In [32]:
user_history_dict = train_data.loc[train_data.rating == 1].groupby('user_index')['item'].apply(list).to_dict()

In [33]:
n_entity, n_relation, kg = load_kg(kg_final)
ripple_set = get_ripple_set(kg, user_history_dict, n_hop=n_hop, n_memory=n_memory)

reading KG file ...
constructing knowledge graph ...
constructing ripple set ...


## Build model

In [ ]:
show_loss = False

In [ ]:
ripple = RippleNet(dim=dim,n_hop=n_hop,
                   kge_weight=kge_weight, l2_weight=l2_weight, lr=lr,
                   n_memory=n_memory,
                   item_update_mode=item_update_mode, using_all_hops=using_all_hops,
                   n_entity=n_entity,n_relation=n_relation)

with tf.Session() as sess:
    model = fit(sess=sess, 
                n_epoch=n_epoch, batch_size=batch_size,n_hop=n_hop,
                model=ripple, train_data=train_data[["user_index", "item", "rating"]].to_numpy(), 
                ripple_set=ripple_set, show_loss=show_loss)
    labels, scores = predict(sess=sess, 
                             batch_size=batch_size, n_hop=n_hop, 
                             model=model, data=test_data[["user_index", "item", "rating"]].to_numpy(),
                             ripple_set=ripple_set)

predictions = [1 if i >= 0.5 else 0 for i in scores]

In [ ]:
test_data['scores'] = scores

In [ ]:
auc_score = auc(test_data, test_data, 
            col_user="user_index",
            col_item="item",
            col_rating="rating",
            col_prediction="scores")

In [ ]:
print("The auc score is {}".format(auc_score))

In [ ]:
acc_score = np.mean(np.equal(predictions, labels))

In [ ]:
print("The acc score is {}".format(acc_score))

In [ ]:
precision_k_score = precision_at_k(test_data, test_data, 
            col_user="user_index",
            col_item="item",
            col_rating="original_rating",
            col_prediction="scores",
            relevancy_method="top_k",
            k=k)

In [ ]:
print("The precision_k_score score at k = {}, is {}".format(k, precision_k_score))

In [ ]:
recall_k_score = recall_at_k(test_data, test_data, 
            col_user="user_index",
            col_item="item",
            col_rating="original_rating",
            col_prediction="scores",
            relevancy_method="top_k",
            k=k)

In [ ]:
print("The recall_k_score score at k = {}, is {}".format(k, recall_k_score))